In [1]:
import requests
from bs4 import BeautifulSoup
import os, psutil
import re
import numpy as np
import pandas as pd
import datetime
import nltk
from nltk.tokenize import sent_tokenize
# nltk.download('punkt_tab')

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm
# from .autonotebook import tqdm as notebook_tqdm

/cluster/home/hlee37/condaenv/torch_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Initialize Model

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Use if models are zipped

# import zipfile

# zip_paths = ['/content/best-sentiment-bert.zip', '/content/sentiment-bert.zip']
# model_paths =[]

# for zip_path in zip_paths:
#   extract_to = zip_path.split('/')[2].split('.')[0]
#   model_paths.append(extract_to)

#   with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#       zip_ref.extractall(extract_to)

In [18]:
model_paths = ["/cluster/home/hlee37/git/gutenberg/phase1/best-relationship-strict-bert", "/cluster/home/hlee37/git/gutenberg/phase1/final-model","/cluster/home/hlee37/git/gutenberg/phase1/sentiment-bert-precision"]

In [19]:
model_list = []

for p in model_paths:
    model = AutoModelForSequenceClassification.from_pretrained(p)
    model.eval()
    model.to(device)
    model_list.append(model)

tokenizer = AutoTokenizer.from_pretrained(model_paths[0])  # shared tokenizer

In [6]:
# def classify_sentence_ensemble(sentence, threshold=0.9):
#     inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     inputs = {k: v.to(device) for k, v in inputs.items()}  # ** important for using GPU

#     with torch.no_grad():
#       probs = []
#       for model in model_list:
#         outputs = model(**inputs)
#         logits = outputs.logits
#         prob = torch.softmax(logits, dim=1)
#         probs.append(prob)

#       avg_probs = torch.mean(torch.stack(probs, dim=0), dim=0)  # keep shape [1, 2]
#       rel_prob = avg_probs[0][1].item()
#       no_rel_prob = avg_probs[0][0].item()

#       predicted_class = 1 if rel_prob >= threshold else 0
    
#     if predicted_class == 0:
#         return None
        
#     return {
#         'sentence': sentence,
#         'predicted_class': predicted_class
#     }


In [20]:
def classify_batch(sentences, threshold=0.9):
    if not sentences:
        return []

    encodings = tokenizer(
        sentences,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        probs_per_model = []
        for model in model_list:
            logits = model(**encodings).logits  # [B, C]
            probs = torch.softmax(logits, dim=1)
            probs_per_model.append(probs)

        avg_probs = torch.mean(torch.stack(probs_per_model), dim=0)  # [B, C]
        rel_probs = avg_probs[:, 1]  # [B]
        preds = (rel_probs >= threshold).long()

    results = []
    for sentence, pred in zip(sentences, preds.cpu().tolist()):
        if pred == 1:
            results.append({
                'sentence': sentence,
                'predicted_class': pred
            })
    return results
 

## Get Text from url links

In [21]:
# cleaning text

def clean_text(raw_text):
    text = raw_text.replace('\n', ' ')
    text = re.sub(r'-\s+', '', text)  # join hyphenated words
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace('—', '-').replace('“', '"').replace('”', '"').replace("’", "'")
    text = re.sub(r'CHAPTER\s[\w\d\.]+.*?(?=\s)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^[A-Z\s\.,\'\-]{10,}$', '', text, flags=re.MULTILINE)
    return text

In [22]:
def get_text(url, diary_url_pd, buffer_size=100):
    url = url.strip()
    now = datetime.datetime.now()

    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Bad Status: {e}, for url: {url}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    all_sentences = []
    buffer = []

    for tag in soup.find_all('p'):
        content = tag.get_text(strip=True)
        if content:
            buffer.append(content)

        if len(buffer) >= buffer_size:
            chunk_text = ' '.join(buffer)
            buffer = []

            cleaned = clean_text(chunk_text)
            sentences = sent_tokenize(cleaned)
            all_sentences.extend(s for s in sentences if len(s.split()) >= 5)

            print(f"🧠 Memory Used: {psutil.Process(os.getpid()).memory_info().rss / 1e6:.2f} MB")

    # Process any remaining text in buffer
    if buffer:
        chunk_text = ' '.join(buffer)
        cleaned = clean_text(chunk_text)
        sentences = sent_tokenize(cleaned)
        all_sentences.extend(s for s in sentences if len(s.split()) >= 5)

    try:
        id_ = diary_url_pd.loc[diary_url_pd['diary'] == url, 'id'].values[0]
        title = diary_url_pd.loc[diary_url_pd['diary'] == url, 'explorer'].values[0]
    except IndexError:
        print(f"❌ Could not find metadata for URL: {url}")
        return

    print(f"✅ Extracted {len(all_sentences)} sentences from {title} (ID: {id_})")
    print(f"⏱️ Time taken: {datetime.datetime.now() - now}")

    return all_sentences


In [10]:
# def main(diary_url_pd):
#     url_list = diary_url_pd['diary'].tolist()
#     diary_url_pd['diary'] = diary_url_pd['diary'].str.strip()

#     for url in tqdm(url_list, desc="📖 Processing Diaries"):
#         try:
#             all_sentences = get_text(url, diary_url_pd)
#             classified_sentences = {}

#             with tqdm(total=len(all_sentences), desc="💿 Classifying Sentences", mininterval=10) as pbar:
#               for i, sentence in enumerate(all_sentences, start=1):
#                   result = classify_sentence_ensemble(sentence)
#                   if result is not None:
#                       classified_sentences[i] = result
#                   pbar.update(1)


#             diary_id = diary_url_pd.loc[diary_url_pd['diary'] == url, 'id'].values[0]
#             diary_title = diary_url_pd.loc[diary_url_pd['diary'] == url, 'explorer'].values[0]

#             df = pd.DataFrame(classified_sentences).T
#             df.to_csv(f"./final/{diary_id}_{diary_title}.csv", index = False)

#         except Exception as e:
#             try:
#                 id_ = diary_url_pd.loc[diary_url_pd['diary'] == url.strip(), 'id'].values[0]
#             except IndexError:
#                 id_ = "unknown"
#             print(f"🔥 Error: {e}, for url: {url}, id: {id_}")

In [23]:
def main(diary_url_pd, batch_size=32):
    url_list = diary_url_pd['diary'].tolist()
    diary_url_pd['diary'] = diary_url_pd['diary'].str.strip()

    for url in tqdm(url_list, desc="📖 Processing Diaries"):
        try:
            all_sentences = get_text(url, diary_url_pd)
            len_sentences=len(all_sentences)
            
            diary_id = diary_url_pd.loc[diary_url_pd['diary'] == url, 'id'].values[0]
            diary_title = diary_url_pd.loc[diary_url_pd['diary'] == url, 'explorer'].values[0]
            
            if len_sentences >= 500000:
                print(f"Number of Sentences exceeds 500k with {len_sentences}")
                
                for chunk_start in range(0, len_sentences, 500000):
                    
                    sentence_batch_num = chunk_start // 500000
                    output_path = f"./final/{diary_id}_{diary_title}_{sentence_batch_num}.csv"
                    if os.path.exists(output_path):
                        print(f"⏩ But {diary_id}_{diary_title}_{sentence_batch_num} exists and skipping.")
                        continue
                    
                    batched_sentences = all_sentences[chunk_start:chunk_start + 500000]
                    classified_sentences = {}
                    i = 1
                    with tqdm(total=len(batched_sentences), desc="💿 Classifying Sentences", mininterval=10) as pbar:
                        for batch_start in range(0, len(batched_sentences), batch_size):
                            batch = batched_sentences[batch_start:batch_start + batch_size]
                            batch_results = classify_batch(batch)
        
                            for res in batch_results:
                                classified_sentences[i] = res
                                i += 1
                            pbar.update(len(batch))
        
                    df = pd.DataFrame(classified_sentences).T
                    df.to_csv(output_path, index=False)
                        
                
                

            else:
                output_path = f"./final/{diary_id}_{diary_title}.csv"
                if os.path.exists(output_path):
                    print(f"⏩ {diary_id}_{diary_title} exists and skipping.")
                    continue
                
                print(f"Number of Sentence Does NOT exceed 500k with {len_sentences}")
                classified_sentences = {}
                i = 1

                with tqdm(total=len_sentences, desc="💿 Classifying Sentences", mininterval=10) as pbar:
                    for start in range(0, len_sentences, batch_size):
                        batch = all_sentences[start:start + batch_size]
                        batch_results = classify_batch(batch)
    
                        for res in batch_results:
                            classified_sentences[i] = res
                            i += 1
    
                        pbar.update(len(batch))
    
    
                df = pd.DataFrame(classified_sentences).T
                df.to_csv(output_path, index=False)

        except Exception as e:
            try:
                id_ = diary_url_pd.loc[diary_url_pd['diary'] == url.strip(), 'id'].values[0]
            except IndexError:
                id_ = "unknown"
            print(f"🔥 Error: {e}, for url: {url}, id: {id_}")


In [12]:
# del diary_pd, diary_url_pd

In [24]:
# read diary list, and list up urls

diary_df = pd.read_csv(os.path.join(os.getcwd(), 'DataForJohn.csv'))
diary_url_df = pd.DataFrame(diary_df[['id', 'Explorer', 'Diary']])
diary_url_df.columns = diary_url_df.columns.str.strip().str.lower()
diary_url_df['diary'] = diary_url_df['diary'].str.strip()

# path_list = [url for url in diary_url_pd['diary'].tolist() if url.startswith('https://gutenberg')]

In [25]:
diary_url_df

,id,explorer,diary
0,1,"Wentworth, Lawson, and Blaxland",https://gutenberg.net.au/ebooks02/0200411h.htm...
1,2,Surveyor G. W. Evans,https://gutenberg.net.au/ebooks13/1300271h.html
2,3,Surveyor General John Oxley,https://gutenberg.net.au/ebooks/e00037.html
3,4,Allan Cunningham,https://gutenberg.net.au/ebooks13/1304431h.html
4,5,Hume and Hovell,https://gutenberg.net.au/ebooks04/0400371h.html
5,6,Capt. Chas. Sturt,https://gutenberg.net.au/ebooks07/0700391h.htm...
6,7,Sir Thomas Mitchell,https://gutenberg.net.au/ebooks/e00035.html
7,8,Sir Thomas Mitchell,https://gutenberg.net.au/ebooks/e00034.html#tr...
8,9,Sir Thomas Mitchell,https://gutenberg.net.au/ebooks/e00036.html
9,10,Edward John Eyre,https://gutenberg.net.au/ebooks/e00048.html


In [16]:
# testing get text with diary 7 for batched run

# diary_7 = diary_url_pd.loc[diary_url_pd['id'] == 7]
# main(diary_7)

In [17]:
# testing get text with diaries


# main(diary_url_pd)

# Classifying Sentences

In [18]:
# # testing the model with diary 1

# diary_1 = diary_url_pd.loc[diary_url_pd['id'] == 1]
# main(diary_1)

In [19]:
# diary_2 = diary_url_pd.loc[diary_url_pd['id'] == 2]
# main(diary_2)

In [20]:
# diary_3 = diary_url_pd.loc[diary_url_pd['id'] == 3]
# main(diary_3)

In [21]:
# main(diary_url_pd)

# Classify in Batch

In [22]:
# url_list = diary_url_pd['diary'].to_list()
# for url in tqdm(url_list, desc="📖 Processing Diaries"):
#     all_sentences = get_text(url, diary_url_pd)

In [23]:
# filtered_df = diary_url_pd[~diary_url_pd['id'].isin([7, 8, 9, 10, 11, 20, 21, 24, 25])]

In [26]:
main(diary_url_df)

📖 Processing Diaries:   3%|███▋                                                                                                                         | 1/34 [00:01<00:36,  1.10s/it]

✅ Extracted 307 sentences from Wentworth, Lawson, and Blaxland (ID: 1)
⏱️ Time taken: 0:00:01.101515
⏩ 1_Wentworth, Lawson, and Blaxland exists and skipping.


📖 Processing Diaries:   6%|███████▎                                                                                                                     | 2/34 [00:02<00:43,  1.37s/it]

🧠 Memory Used: 3171.02 MB
🧠 Memory Used: 3171.02 MB
🧠 Memory Used: 3171.02 MB
🧠 Memory Used: 3171.02 MB
✅ Extracted 701 sentences from Surveyor G. W. Evans (ID: 2)
⏱️ Time taken: 0:00:01.558244
⏩ 2_Surveyor G. W. Evans exists and skipping.


📖 Processing Diaries:   9%|███████████                                                                                                                  | 3/34 [00:04<00:45,  1.48s/it]

...

⏩ 31_D. Lindsay exists and skipping.


📖 Processing Diaries:  94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋       | 32/34 [19:39<00:22, 11.12s/it]

🧠 Memory Used: 3294.75 MB
🧠 Memory Used: 3294.75 MB
🧠 Memory Used: 3294.75 MB
🧠 Memory Used: 3294.75 MB
🧠 Memory Used: 3294.75 MB
🧠 Memory Used: 3294.75 MB
🧠 Memory Used: 3294.75 MB
🧠 Memory Used: 3294.75 MB
🧠 Memory Used: 3294.75 MB
✅ Extracted 4252 sentences from Hon. D.C. Carnegie (ID: 32)
⏱️ Time taken: 0:00:06.994786
⏩ 32_Hon. D.C. Carnegie exists and skipping.


📖 Processing Diaries: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [19:46<00:00, 34.89s/it]

✅ Extracted 12 sentences from F. Hann (ID: 33)
⏱️ Time taken: 0:00:06.589761
⏩ 33_F. Hann exists and skipping.
Bad Status: Invalid URL 'no diary-please search if there is anything in your library': No scheme supplied. Perhaps you meant https://no diary-please search if there is anything in your library?, for url: no diary-please search if there is anything in your library
🔥 Error: object of type 'NoneType' has no len(), for url: no diary-please search if there is anything in your library, id: 34


# Check files

In [2]:
import glob 

all_df = []
folder_path = '/cluster/home/hlee37/git/gutenberg/phase2/final'
file_names = '*.csv'

file_paths= glob.glob(os.path.join(folder_path, file_names))

for file in file_paths:
    try:
        df=pd.read_csv(file)
        df['file_name']= file.split('/')[8].split('.')[0]
        all_df.append(df)
        print(f"success opened: {file}")
    except Exception as e:
        print(f"failed: {file}, reason: {e}")


combined_df=pd.concat(all_df, ignore_index=True)

success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_0.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_1.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_10.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_11.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_2.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_3.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_4.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_5.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_6.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_7.csv
success opened: /cluster/home/hlee37/git/gutenberg/phase2/final/10_Edward John Eyre_8.csv
success 

In [3]:
combined_df.head()

,sentence,predicted_class,file_name
0,In offering to the public an account of Expedi...,1,10_Edward John Eyre_0
1,"After starting, we proceeded across a high bar...",1,10_Edward John Eyre_0
2,"33 degrees 14 minutes S, is a practicable pass...",1,10_Edward John Eyre_0
3,At night too after dark we found that a party ...,1,10_Edward John Eyre_0
4,"By digging in the gravelly bed of the channel,...",1,10_Edward John Eyre_0


In [4]:
pd.set_option('display.max_rows', 100)
combined_df['explorer_name'] = combined_df.loc[:,'file_name'].str.split('_').str[1]
combined_df.groupby('explorer_name').count().head(67)[['sentence']].sort_values(by='sentence', ascending=False)

,sentence
explorer_name,
Sir Thomas Mitchell,851249
Edward John Eyre,783985
Ernest Giles,215016
Ludwig Leichhardt,185956
McDouall Stuart,150393
Burke & Wills,69034
John & A,58283
Frank & Alexander Jardine,12715
Hon,238


In [5]:
df_stats = combined_df.groupby('explorer_name').count().head(67)[['sentence']].sort_values(by='sentence', ascending=False)

In [6]:
dup_counts = (
    combined_df[combined_df.duplicated()]
    .groupby('explorer_name')
    .count()['sentence']
    .rename('num_duplicates')
)

# Step 2: Join with df_stats
df_stats = df_stats.join(dup_counts, how='left')

In [7]:
df_stats['duplicate_pct']=df_stats['num_duplicates']/df_stats['sentence']

In [11]:
combined_df.groupby('explorer_name')['sentence'].nunique().sort_values()

explorer_name
Allan Cunningham                                  2
Wentworth, Lawson, and Blaxland                   9
Edmund B                                          9
Surveyor G                                       11
Hume and Hovell                                  22
W                                                34
Capt                                             78
Frank & Alexander Jardine                        95
Major Warburton                                  98
In search of Burke & Wills—Frederick Walker     106
Surveyor General John Oxley                     106
Burke & Wills                                   129
A                                               151
John & A                                        203
D                                               216
Hon                                             238
McDouall Stuart                                 280
Ludwig Leichhardt                               369
Ernest Giles                                    49

In [12]:
unique_df=combined_df.drop_duplicates()

In [16]:
unique_df.groupby('explorer_name')['sentence'].count().sort_values(ascending=False)

explorer_name
Edward John Eyre                               6215
Sir Thomas Mitchell                            4735
Ernest Giles                                   2231
McDouall Stuart                                1551
Ludwig Leichhardt                              1166
Burke & Wills                                   330
Hon                                             238
D                                               216
John & A                                        203
A                                               151
Surveyor General John Oxley                     106
In search of Burke & Wills—Frederick Walker     106
Major Warburton                                  98
Frank & Alexander Jardine                        95
Capt                                             78
W                                                34
Hume and Hovell                                  22
Surveyor G                                       11
Edmund B                                          